In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.utils import timeseries_dataset_from_array
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("jena_climate_2009_2016.csv")

In [3]:
temperature = df["T (degC)"].values

In [4]:
raw_data = df.loc[:, "p (mbar)":"wd (deg)"].values

**Data split**

In [5]:
num_train_samples = int(0.5 * len(raw_data))
num_val_samples = int(0.25 * len(raw_data))
num_test_samples = len(raw_data) - num_train_samples - num_val_samples

**Normalizing the data**

In [6]:
mean = raw_data[:num_train_samples].mean(axis=0)
raw_data -= mean

std = raw_data[:num_train_samples].std(axis=0)
raw_data /= std

**Instantiating datasets for training, validation, and testing**

In [7]:
sampling_rate = 6
sequence_length = 120
delay = sampling_rate * (sequence_length + 24 - 1)
batch_size = 256

In [8]:
train_dataset = timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=0,
    end_index=num_train_samples)

val_dataset = timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=num_train_samples,
    end_index=num_train_samples + num_val_samples)

test_dataset = timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=num_train_samples + num_val_samples)

### Stacking recurrent layers

**Training and evaluating a dropout-regularized, stacked GRU model**

In [9]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)

In [13]:
inputs = Input(shape=(sequence_length, raw_data.shape[-1]))
x = GRU(32, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = GRU(32, recurrent_dropout=0.5)(x)
x = Dropout(0.5)(x)
outputs = Dense(1)(x)
model = Model(inputs, outputs)

callbacks = [ModelCheckpoint("jena_stacked_gru_dropout.keras", save_best_only=True)]

In [14]:
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset, epochs=5, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/5
819/819 [==============================] - 680s 827ms/step - loss: 24.7900 - mae: 3.6780 - val_loss: 9.5977 - val_mae: 2.3931
Epoch 2/5
819/819 [==============================] - 683s 834ms/step - loss: 14.0631 - mae: 2.9055 - val_loss: 9.7589 - val_mae: 2.4207
Epoch 3/5
819/819 [==============================] - 646s 789ms/step - loss: 13.2798 - mae: 2.8213 - val_loss: 9.6677 - val_mae: 2.3981
Epoch 4/5
819/819 [==============================] - 641s 783ms/step - loss: 12.8527 - mae: 2.7797 - val_loss: 9.4138 - val_mae: 2.3638
Epoch 5/5
819/819 [==============================] - 643s 785ms/step - loss: 12.2973 - mae: 2.7191 - val_loss: 8.7969 - val_mae: 2.2881


In [15]:
model = load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

405/405 [==============================] - 37s 89ms/step - loss: 9.6665 - mae: 2.4366
Test MAE: 2.44
